<a href="https://colab.research.google.com/github/Nagmashaik123/Section203-Group06/blob/main/Countries_Cities_Location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_csv(r"CovidVaccine.csv",error_bad_lines=False)
df

In [36]:
!pip install geopy

In [37]:
import numpy as np
import re

# Reading emoji patterns which contains emoticons , symbols & pictographs, transport & map symbols , flags
emojisCollection = re.compile("["
                       u"\U0001F600-\U0001F64F"  
                       u"\U0001F300-\U0001F5FF"  
                       u"\U0001F680-\U0001F6FF"  
                       u"\U0001F1E0-\U0001F1FF"  
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       "]+", flags=re.UNICODE)
df['TweetLocation'] = (df['TweetLocation'].str.replace(emojisCollection, '').str.strip())

df['TweetLocation'] = df.TweetLocation.str.split(', ').str.get(0)
df['TweetLocation'] = df['TweetLocation'].replace(np.nan, 'Unknown', regex=True)
df['TweetLocation']

0                     San Antonio
1                         Unknown
2                         Unknown
3                          London
4                        Montreal
5                   United States
6                       Las Vegas
7                      Georgetown
8                        Portland
9                      Pittsburgh
10                        Unknown
11                      Gateshead
12                         Indore
13                        England
14                        Unknown
15                      Frankfort
16    Royaume-Uni. United Kingdom
17                         Canada
18                        Unknown
19                   Winter Haven
20                      Beckenham
21                        Unknown
22                         Orange
23                    District 13
24                     Washington
Name: TweetLocation, dtype: object

In [51]:
from functools import partial
from geopy.geocoders import Nominatim
import re

countryList =[]
def checkCountry(city):
  geolocator = Nominatim(user_agent="COVID-19 vaccination attitudes")
  geocode = partial(geolocator.geocode, language="en")
  if city != "Unknown" and not re.match(r'^[_\W]+$', city)  :
    countries = geocode(city)   
    if countries.find(", ") != -1:      
      country = str(countries).split(", ")[-1]
      countryList.append(country) 
    elif countries is None:
      countryList.append("Unknown")    
    else:
      countryList.append(countries)          
  else:
    countryList.append("Unknown")
    return countryList
    

In [ ]:
cities=[]
countries=[]
cities = df['TweetLocation'].tolist()
for city in cities:
    try:
        countries.append(checkCountry(city))  #Calling the checkCountry function
    except:
        countries.append('Excetion Raised while retriving Country')

In [18]:
dict_1=dict(zip(cities,countries))

In [ ]:
df['Country']=df['TweetLocation'].map(dict_1)
df['Country']